### Match com equipe e Upload na Turn

A partir dos usuários selecionados para o dia no Passo 1, une suas informações com suas informações de equipe/estabelecimentos que esta no Big Query e advem da Turn e após isso da upload na turn.

#### Configurações iniciais do ambiente

#### Match

#### Upload

In [ ]:
tokens_municipios = [
    {"municipio": "Paulo Ramos", "id_sus": "210810", "token": os.getenv('ENV_PAULORAMOS_MA')},
    {"municipio": "Pacoti", "id_sus": "210810", "token": os.getenv('ENV_PACOTI_CE')},
    {"municipio": "Marajá do Sena", "id_sus": "210810", "token": os.getenv('ENV_MARAJADOSENA_MA')},
    {"municipio": "Monsenhor Tabosa", "id_sus": "210810", "token": os.getenv('ENV_MONSENHORTABOSA_CE')},
    {"municipio": "Lago Verde", "id_sus": "210590", "token": os.getenv('ENV_LAGOVERDE_MA')}                                                 
]

In [ ]:
for linha in dataframe:
    